In [6]:
import shutil
from decimal import Decimal
from pathlib import Path

import pandas as pd

from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model import BarType, InstrumentId, instruments, BarSpecification, Venue
from nautilus_trader.model.enums import BarAggregation, PriceType, AssetClass, OmsType, AccountType
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.model import Symbol
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Bar

In [7]:
catalog = ParquetDataCatalog("./catalog")
nasdaq_venue = Venue("NASDAQ")

# available_instruments = {instrument.id.symbol.value for instrument in catalog.instruments()}
# if "SH" not in available_instruments and "VOO" not in available_instruments:
#     raise RuntimeError("Required instruments not found")

# for inst in catalog.instruments():
#     instrument_id = inst.id
#     catalog.bars(
#         [BarType(
#             instrument_id,
#             BarSpecification(1, BarAggregation.MINUTE, PriceType.LAST)
#         )]
#     )

start_time = "2024-04-01"
end_time = "2024-06-01"


data_configs = [
    # 1 min
    BacktestDataConfig(
        catalog_path="./catalog",
        data_cls=Bar,
        instrument_id=InstrumentId.from_str("VOO.NASDAQ"),
        bar_spec=BarSpecification(
            1, BarAggregation.MINUTE, PriceType.LAST
        ),
        start_time=start_time,
        end_time=end_time
    ),
    # 30 min
    BacktestDataConfig(
        catalog_path="./catalog",
        data_cls=Bar,
        instrument_id=InstrumentId.from_str("VOO.NASDAQ"),
        bar_spec=BarSpecification(
            30, BarAggregation.MINUTE, PriceType.LAST
        ),
        start_time=start_time,
        end_time=end_time
    ),
    # 1 day
    BacktestDataConfig(
        catalog_path="./catalog",
        data_cls=Bar,
        instrument_id=InstrumentId.from_str("VOO.NASDAQ"),
        bar_spec=BarSpecification(
            1, BarAggregation.DAY, PriceType.LAST
        ),
        start_time=start_time,
        end_time=end_time
    )
]

In [8]:
venue_conf = BacktestVenueConfig(
    "NASDAQ",
    oms_type="HEDGING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"],
    # fill_model=FillModel(),
    bar_adaptive_high_low_ordering=True,
)

In [9]:
bt_config = BacktestEngineConfig(
    trader_id="Simple-001",
    logging=LoggingConfig("WARNING"),
    # strategies=[

    #     ImportableStrategyConfig(
    #     strategy_path="breakoutv2:BreakoutV2",
    #     config_path="breakoutv2:BreakoutV2Config",
    #     config={
    #         "main_symbol": InstrumentId.from_str("VOO.NASDAQ"),
    #         "reverse_symbol": InstrumentId.from_str("SH.NASDAQ"),
    #         "long_entry": 1,
    #         "short_entry": 1,
    #         "long_exit": 7,
    #         "short_exit": 7,
    #         "ema_lookback_hours": 50,
    #     },
    #     ),
    # ],
    strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config={
            "instrument_id": InstrumentId.from_str("VOO.NASDAQ"),
            "bar_type": BarType(
                InstrumentId.from_str("VOO.NASDAQ"),
                BarSpecification(1, BarAggregation.MINUTE, PriceType.LAST)),
            "fast_ema_period": 10,
            "slow_ema_period": 20,
            "trade_size": Decimal(1_000_000),
        },
    ),
]
)

In [10]:
config = BacktestRunConfig(
    engine=bt_config,
    data=data_configs,
    venues=[venue_conf],
)

node = BacktestNode(configs=[config])

results = node.run()

2024-04-01T00:00:00.000000000Z [WARN] Simple-001.EMACross: Received <Bar[0]> data for unknown bar type
2024-04-08T14:00:00.000000000Z [WARN] Simple-001.EMACross: Bar OHLC is single price; implies no market information
2024-04-08T14:00:00.000000000Z [WARN] Simple-001.EMACross: Bar OHLC is single price; implies no market information
2024-04-08T14:00:00.000000000Z [WARN] Simple-001.EMACross: Bar OHLC is single price; implies no market information
2024-04-11T14:51:00.000000000Z [WARN] Simple-001.EMACross: Bar OHLC is single price; implies no market information
2024-04-11T14:51:00.000000000Z [WARN] Simple-001.EMACross: Bar OHLC is single price; implies no market information
2024-04-11T14:51:00.000000000Z [WARN] Simple-001.EMACross: Bar OHLC is single price; implies no market information


KeyboardInterrupt: 

In [ ]:
results[0].stats_pnls

In [ ]:
obj = catalog.bars(
    [BarType(
        InstrumentId.from_str("VOO.NASDAQ"),
        BarSpecification(1, BarAggregation.MINUTE, PriceType.LAST)
    )]
)[0]

print("Accessible properties/fields:")
for attr in dir(obj):
    if not attr.startswith("__"):
        print(attr)